<a href="https://colab.research.google.com/github/EN10/BabyLlama/blob/main/Baby_Llama_4K.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/karpathy/llama2.c.git

Cloning into 'llama2.c'...
remote: Enumerating objects: 1434, done.
remote: Total 1434 (delta 0), reused 0 (delta 0), pack-reused 1434
Receiving objects: 100% (1434/1434), 1.17 MiB | 2.16 MiB/s, done.
Resolving deltas: 100% (871/871), done.


In [2]:
pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.9 MB/s eta 0:00:00


In [3]:
# runtime 100s
!cd llama2.c && python tinystories.py download

data/TinyStories_all_data.tar.gz: 100% 1.50G/1.50G [00:23<00:00, 69.2MiB/s]
Unpacking data/TinyStories_all_data.tar.gz...
Download done.
Number of shards: 50
Example story:
{'story': '\n\nLily and Ben are friends. They like to play in the park. One day, they see a big tree with a swing. Lily wants to try the swing. She runs to the tree and climbs on the swing.\n"Push me, Ben!" she says. Ben pushes her gently. Lily feels happy. She swings higher and higher. She laughs and shouts.\nBen watches Lily. He thinks she is cute. He wants to swing too. He waits for Lily to stop. But Lily does not stop. She swings faster and faster. She is having too much fun.\n"Can I swing too, Lily?" Ben asks. Lily does not hear him. She is too busy swinging. Ben feels sad. He walks away.\nLily swings so high that she loses her grip. She falls off the swing. She lands on the ground. She hurts her foot. She cries.\n"Ow, ow, ow!" she says. She looks for Ben. She wants him to help her. But Ben is not there. He is 

In [4]:
# !cd llama2.c && python tinystories.py train_vocab --vocab_size=4096

!cd ./llama2.c/data && wget https://huggingface.co/datasets/enio/TinyStories/resolve/main/tok4096/tok4096.vocab
!cd ./llama2.c/data && wget https://huggingface.co/datasets/enio/TinyStories/resolve/main/tok4096/tok4096.model

# !cd llama2.c && python tinystories.py pretokenize --vocab_size=4096
!cd ./llama2.c/data && mkdir tok4096
!cd ./llama2.c/data/tok4096 && wget https://huggingface.co/datasets/enio/TinyStories/resolve/main/tok4096/TinyStories_tok4096.tar.gz
!cd ./llama2.c/data/tok4096 && tar -xf TinyStories_tok4096.tar.gz

--2023-12-11 19:02:27--  https://huggingface.co/datasets/enio/TinyStories/resolve/main/tok4096/tok4096.vocab
Resolving huggingface.co (huggingface.co)... 18.164.174.55, 18.164.174.118, 18.164.174.17, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.55|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53534 (52K) [text/plain]
Saving to: ‘tok4096.vocab’

tok4096.vocab       100%[===================>]  52.28K  --.-KB/s    in 0.06s   

2023-12-11 19:02:27 (808 KB/s) - ‘tok4096.vocab’ saved [53534/53534]

--2023-12-11 19:02:27--  https://huggingface.co/datasets/enio/TinyStories/resolve/main/tok4096/tok4096.model
Resolving huggingface.co (huggingface.co)... 18.164.174.55, 18.164.174.23, 18.164.174.118, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.55|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/65/c8/65c8615bfd6b6d94c8e62381ee6653578b439f71ff04b86aaefa9828dec8bb70/64

In [5]:
!cd llama2.c && python tokenizer.py --tokenizer-model=data/tok4096.model

In [6]:
# change bfloat16 to float16 on line 72
!sed -i "72s/.*/dtype = 'float16'  # float32|bfloat16|float16/" ./llama2.c/train.py

In [7]:
!cd llama2.c && python train.py --vocab_source=custom --vocab_size=4096 --compile=False \
  --dim=64 --batch_size=16 --always_save_checkpoint=True --eval_interval=1000 --max_iters=1001 \
  #--init_from='resume'"

Overriding: vocab_source = custom
Overriding: vocab_size = 4096
Overriding: compile = False
Overriding: dim = 64
Overriding: batch_size = 16
Overriding: always_save_checkpoint = True
Overriding: eval_interval = 1000
Overriding: max_iters = 1001
tokens per iteration will be: 16,384
breaks down as: 4 grad accum steps * 1 processes * 16 batch size * 256 max seq len
Initializing a new model from scratch
num decayed parameter tensors: 43, with 575,488 parameters
num non-decayed parameter tensors: 13, with 832 parameters
using fused AdamW: True
Created a PretokDataset with rng seed 42
Created a PretokDataset with rng seed 42
Created a PretokDataset with rng seed 42
step 0: train loss 8.3381, val loss 8.3380
0 | loss 8.3382 | lr 0.000000e+00 | 4630.69ms | mfu -100.00%
1 | loss 8.3323 | lr 5.000000e-07 | 93.44ms | mfu -100.00%
2 | loss 8.3353 | lr 1.000000e-06 | 96.20ms | mfu -100.00%
3 | loss 8.3403 | lr 1.500000e-06 | 92.14ms | mfu -100.00%
4 | loss 8.3384 | lr 2.000000e-06 | 98.49ms | mfu -

In [8]:
!cd llama2.c && gcc -O3 -o run run.c -lm

In [9]:
!cd ./llama2.c && ./run out/model.bin -z data/tok4096.bin -t 0.8 -n 256 -i "Once upon a time "

Once upon a time  calledel off the manyrag ex doilliant toug meatummy valuable nap adventuresf B prize slide treats flower never Luna have thoughtairsel teach park  One sandwiches repair sign sy little another otter Jerry girlarl Onesrow woman ground strawberry Timmy hippo sheep story bathroom cool&lingir may turn by ran TheyWhatsh rock croions witht droveloo Daisyart nosy kid is Snowy
achieved tok/s: 1064.935065
